In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("covid_symptoms.csv")

In [3]:
df.head()

,age,gender,vaccination_status,fever,cough,fatigue,shortness_of_breath,loss_of_smell,headache,diabetes,hypertension,heart_disease,asthma,cancer,hospitalized,icu_admission,mortality
0,51,Male,Unvaccinated,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,92,Male,Fully Vaccinated,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,14,Male,Fully Vaccinated,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,71,Male,Fully Vaccinated,1,1,0,0,0,0,0,1,0,0,0,0,0,0
4,60,Male,Unvaccinated,0,1,1,1,1,0,1,1,1,0,0,1,1,0


In [5]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [6]:
df['covid'] = (
    (df['fever'] == 1) |
    (df['cough'] == 1) &
    (df['loss_of_smell'] == 1)
).astype(int)

In [8]:
X = df[['fever', 'cough', 'shortness_of_breath', 'loss_of_smell', 'headache']]
y = df['covid']

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size =0.3)

In [11]:
model = LGBMClassifier(n_estimators=100, max_depth = 3, learning_rate=0.1)
model.fit(X_train,y_train)

[LightGBM] [Info] Number of positive: 723, number of negative: 1377
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000560 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10
[LightGBM] [Info] Number of data points in the train set: 2100, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.344286 -> initscore=-0.644253
[LightGBM] [Info] Start training from score -0.644253
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

LGBMClassifier(max_depth=3)

In [13]:
y_pred = model.predict(X_test)

In [14]:
from sklearn.metrics import accuracy_score, classification_report
print(f"accuracy score: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

accuracy score: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       558
           1       1.00      1.00      1.00       342

    accuracy                           1.00       900
   macro avg       1.00      1.00      1.00       900
weighted avg       1.00      1.00      1.00       900



In [17]:
## New patient
new_pat = pd.DataFrame([{'fever' : 0, 
                        'cough' : 0, 
                        'shortness_of_breath' : 1, 
                        'loss_of_smell' : 1, 
                        'headache' : 1
                        }])

if model.predict(new_pat) == 1:
    print("Covid +ve")

else:
    print("Covid -ve")

Covid -ve
